In [39]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import scipy as sc
from sklearn import preprocessing
from sklearn import model_selection

from recsys.preprocess import *
from recsys.utility import *

RANDOM_STATE = 2342

%matplotlib inline

In [40]:
train = pd.read_csv('data/train_final.csv', delimiter='\t')
playlists = pd.read_csv('data/playlists_final.csv', delimiter='\t')
tracks = pd.read_csv('data/tracks_final.csv', delimiter='\t')

target_playlists = pd.read_csv('data/target_playlists.csv', delimiter='\t')
target_tracks = pd.read_csv('data/target_tracks.csv', delimiter = '\t')

In [41]:
len(train)

1040522

In [42]:
train.head()

,playlist_id,track_id
0,3271849,2801526
1,5616275,727878
2,11267488,2805283
3,10103900,1515105
4,3836898,2945623


In [46]:
train, test, target_playlists, target_tracks = train_test_split(train, test_size=0.3, min_playlist_tracks=7)

In [47]:
train.head()

,playlist_id,track_id
0,3271849,2801526
1,5616275,727878
2,11267488,2805283
3,10103900,1515105
4,3836898,2945623


In [48]:
len(train)

1001392

In [49]:
test.head()

,playlist_id,track_id
118.0,10083808,1989513
171.0,6331847,3639589
183.0,6425089,303587
200.0,7944034,1595978
228.0,7906617,1531731


In [50]:
len(test)

39130

In [51]:
target_playlists

array([ 5881685,  3773136, 10038521, ...,  4040480,  8433581,  5600317])

In [52]:
len(target_playlists)

7826

In [53]:
target_tracks

array([  3.94000000e+02,   6.67000000e+02,   1.36500000e+03, ...,
         5.00735400e+06,   5.01470200e+06,   5.01827400e+06])

In [54]:
len(target_tracks)

26953

In [55]:
# most popular tracks: the ones we want to recommend
most_popular = get_most_popular_tracks(train)

# we don't want to recommend tracks already in the playlists...
tracks_in_playlist = get_playlist_track_list(train)

In [70]:
tracks_to_suggest = most_popular.index.values

In [78]:
target_playlists

array([ 5881685,  3773136, 10038521, ...,  4040480,  8433581,  5600317])

In [79]:
predictions = {}
for pl_id in target_playlists:
    count = 0
    i = 0
    pred = []
    while count < 5:
        if tracks_to_suggest[i] not in tracks_in_playlist[pl_id]:
            # Predict track i
            # IMPORTANT: should we check if the track to suggest is in target_tracks?
            pred.append(tracks_to_suggest[i])
            count += 1
        i += 1
    predictions[pl_id] = pred

In [80]:
predictions

{5881685: [1563309, 1363985, 3705881, 1595978, 3779477],
 3773136: [1563309, 1363985, 3705881, 1595978, 3779477],
 10038521: [1563309, 1363985, 3705881, 1595978, 3779477],
 7455024: [1563309, 1363985, 3705881, 1595978, 3779477],
 5788943: [1563309, 1363985, 3705881, 1595978, 3779477],
 8920509: [1563309, 1363985, 3705881, 1595978, 3779477],
 11185169: [1563309, 1363985, 3705881, 1595978, 3779477],
 10936231: [1563309, 1363985, 3705881, 1595978, 3779477],
 1767988: [1563309, 1363985, 3705881, 1595978, 3779477],
 2962399: [1563309, 1363985, 3705881, 1595978, 3779477],
 1769902: [1563309, 1363985, 3705881, 1595978, 3779477],
 2948215: [1563309, 1363985, 3705881, 1595978, 3779477],
 3258194: [1563309, 1363985, 3705881, 1595978, 3779477],
 5404847: [1563309, 1363985, 3705881, 1595978, 3779477],
 10483130: [1563309, 1363985, 3705881, 1595978, 3779477],
 8523879: [1563309, 1363985, 3705881, 1595978, 3779477],
 10058437: [1563309, 1363985, 3705881, 1595978, 3779477],
 7541606: [1563309, 136398

In [82]:
evaluate(test, predictions)

0.0011891983985007231